In [1]:
from gliner import GLiNER
from contextlib import redirect_stdout
import pandas as pd
import spacy

nlp = spacy.load("en_core_web_sm")
with redirect_stdout(None):
    model = GLiNER.from_pretrained("urchade/gliner_mediumv2.1")
    model.eval()

c:\Users\abhin\miniconda3\envs\nlpEnv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def get_objects(text):
    doc = nlp(text)
    labels = ["ORGANIZATION"]
    entities = model.predict_entities(text, labels, threshold=0.4)
    my_dict = {}

    for ent in entities:
        my_dict.setdefault(f"gliner_{ent['label']}", []).append(ent["text"])
    
    for ent in doc.ents:
        my_dict.setdefault(f"spacy_{ent.label_}", []).append(ent.text)
    
    return pd.Series(my_dict)

In [4]:
df = pd.read_csv("b2.csv")
df["Sentences"] = df.Sentences.apply(lambda x:x.lstrip().replace(";", ","))
df = pd.concat([df, df.Sentences.apply(get_objects)], axis=1)
df

,Sentences,gliner_ORGANIZATION,spacy_ORG,spacy_MONEY,spacy_PERSON,spacy_FAC,spacy_LOC,spacy_CARDINAL,spacy_NORP,spacy_GPE,spacy_PRODUCT,spacy_TIME,spacy_DATE
0,"₹1,200 on Flipkart: Wireless headphones, phone...","[Flipkart, Flipkart]",[Flipkart],"[1,200]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"$15 on Amazon: Coffee beans, subscription box ...","[Amazon, Amazon]","[Amazon, Amazon]","[15, 15]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"$40 at IKEA: Picture frame, throw pillows (IKE...","[IKEA, IKEA]",[IKEA],"[40, 40]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"€20 at H&M: T-shirt, pair of jeans (H&M, €20).","[H&M, H&M]","[H&M, H&M]",[20],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"$75 on Nike.com: Running shoes, athletic socks...","[Nike.com, Nike.com]",[Nike.com],"[75, 75]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
190,$80 on Gaia: Subscription for online yoga and ...,"[Gaia, Gaia]",[Gaia: Subscription],"[80, 80]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191,"₩40,000 at Local Craft Market: Handmade soap, ...","[Local Craft Market, Local Craft Market]",[Local Craft Market:],"[40,000, 40,000]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
192,$70 on Skillshare: Subscription for online lea...,"[Skillshare, Skillshare]",[Skillshare],"[70, 70]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193,₹500 on Swiggy: Order groceries for the week (...,[Swiggy: Order groceries for the week (Swiggy],[Swiggy],"[500, 500]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[the week]


In [5]:
df[df.gliner_ORGANIZATION.isna()]

,Sentences,gliner_ORGANIZATION,spacy_ORG,spacy_MONEY,spacy_PERSON,spacy_FAC,spacy_LOC,spacy_CARDINAL,spacy_NORP,spacy_GPE,spacy_PRODUCT,spacy_TIME,spacy_DATE
14,"$90 on [invalid URL removed]: Gym bag, water b...",NaN,NaN,"[90, 90]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
